<a href="https://colab.research.google.com/github/zahrahkkn-lab/operational_urban_developpement/blob/main/C%C3%B3pia_de_yolo_cars_video.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip -q install ultralytics yt-dlp opencv-python

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 180.3/180.3 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 31.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 67.4 MB/s eta 0:00:00


In [2]:
import subprocess, glob, os

LIVE_URL = "https://www.youtube.com/watch?v=bNaSlizKHEM"

# grava a partir do ponto atual do live
subprocess.run([
    "yt-dlp",
    "--download-sections", "*00:00-00:00:05",  # 60s
    "-f", "bv*[ext=mp4]/b",
    "-o", "/content/live_clip.%(ext)s",
    LIVE_URL
], check=True)

files = glob.glob("/content/live_clip.*")
print("Downloaded clip:", files)
VIDEO_PATH = files[0]


Downloaded clip: ['/content/live_clip.mp4']


In [3]:
from ultralytics import YOLO
import os, torch

VIDEO_PATH = "/content/live_clip.mp4"
if not os.path.exists(VIDEO_PATH):
    raise FileNotFoundError(VIDEO_PATH)

model = YOLO("yolov8n.pt")

use_half = torch.cuda.is_available()  # half só faz sentido na GPU

for _ in model.predict(
    source=VIDEO_PATH,
    classes=[2],
    conf=0.30,
    imgsz=480,
    vid_stride=2,
    half=use_half,
    stream=True,
    verbose=False,
    save=True,
    project="/content",
    name="yolo_live_cars",
):
    pass

print("Output folder: /content/yolo_live_cars/")


Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
Results saved to /content/yolo_live_cars
Output folder: /content/yolo_live_cars/


In [4]:
import glob
from base64 import b64encode
from IPython.display import HTML

video_files = glob.glob("/content/yolo_live_cars/live_clip.mp4")

if video_files:
    out_vid = video_files[0]
    mp4 = open(out_vid, "rb").read()
    data_url = "data:video/mp4;base64," + b64encode(mp4).decode()

    display(HTML(f"""
    <video width="900" controls>
      <source src="{data_url}" type="video/mp4">
    </video>
    """))
else:
    print("Erro: O vídeo processado 'live_clip.mp4' não foi encontrado em /content/yolo_live_cars/.")
    print("Isso geralmente acontece se o download do vídeo original ou o processamento pelo modelo YOLOv8 não foi bem-sucedido.")
    print("Por favor, verifique as células anteriores para garantir que o download e a detecção foram concluídos sem erros.")
    # Optionally, you can list the contents of the directory for debugging
    # import os
    # print(f"Conteúdo de /content/yolo_live_cars/: {os.listdir('/content/yolo_live_cars/')}")

Erro: O vídeo processado 'live_clip.mp4' não foi encontrado em /content/yolo_live_cars/.
Isso geralmente acontece se o download do vídeo original ou o processamento pelo modelo YOLOv8 não foi bem-sucedido.
Por favor, verifique as células anteriores para garantir que o download e a detecção foram concluídos sem erros.
